Trabajo con la Metadata de la Base de Datos
====

* Última modificación: 12 de septiembre de 2022.

Preparación
----

In [1]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=True,
    future=True,
)

Creación del objecto Metadata
---

In [2]:
#
# Metadata son los objetos de Python que represetan la estructura de la base de datos
#
from sqlalchemy import MetaData

metadata_obj = MetaData()

Creación de una tabla
---

In [3]:
from sqlalchemy import Column, Integer, String, Table

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

Acceso a las columnas de la tabla
----

In [4]:
#
# Extracción del nombre de un elemento.
# Table.c es un objeto de Python que representa la colección de columnas de la tabla
#
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [5]:
#
# Claves de la colección de columnas de la tabla
#
user_table.c.keys()

['id', 'name', 'fullname']

Declaración de restricciones simples
----

In [6]:
#
# Información sobre la clave primaria
#
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [7]:
#
# Creación de una tabla con una clave foranea
#
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),  # Clave foranea
    Column("email_address", String, nullable=False),
)

Emisión del DDL a la Base de Datos
----

In [8]:
metadata_obj.create_all(engine)

2022-09-13 11:55:55,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:55,716 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-13 11:55:55,717 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:55,718 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-13 11:55:55,718 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:55,719 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-13 11:55:55,720 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:55,720 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-13 11:55:55,721 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:55,722 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-13 11:55:55,723 INFO sqlalchemy.engine.Engine [no key 0.00058s] ()
2022-09-13 11:55:55,724 INFO sqlalchemy.engine.Engine 
C

In [9]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    print(result.all())

2022-09-13 11:55:55,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:55,802 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table';
2022-09-13 11:55:55,803 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ()
[('user_account',), ('address',)]
2022-09-13 11:55:55,804 INFO sqlalchemy.engine.Engine ROLLBACK


Definición de la Metadata de Tablas con el ORM
----

In [10]:
#
# En el modelo ORM la metadata es guardada en el objetto registry
#
from sqlalchemy.orm import registry

mapper_registry = registry()
mapper_registry.metadata

MetaData()

In [11]:
#
# Los objetos de la base de datos son manipualdos a traves de una base indirecta
#
Base = mapper_registry.generate_base()

**Nota**

In [12]:
#
# Los dos pasos anteriores se combinan en una sola línea
#
from sqlalchemy.orm import declarative_base

Base = declarative_base()

Declaración de clases mapeadas
---

In [13]:
from sqlalchemy.orm import relationship


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [14]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

Detalles
----

In [15]:
#
# Las clases tienen un método __init__() generado automáticamente
#
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [16]:
#
# El método __repr__() es generado automáticamente y es completamente opcional
#
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

Envio del DDL a la base de datos
----

In [17]:
#
# Emite la declaración CREATE dado el registro ORM
#
mapper_registry.metadata.create_all(engine)

#
# El objeto Metadata tambien está presente en la base declarativa
#
Base.metadata.create_all(engine)

2022-09-13 11:55:56,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:56,273 INFO sqlalchemy.engine.Engine COMMIT
2022-09-13 11:55:56,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:56,275 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-13 11:55:56,275 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:56,276 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-13 11:55:56,276 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:56,277 INFO sqlalchemy.engine.Engine COMMIT


Forma alternativa de combinación de las declaraciones y el ORM
---

In [18]:
mapper_registry = registry()
Base = mapper_registry.generate_base()


class User(Base):
    __table__ = user_table
    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"


class Address(Base):
    __table__ = address_table
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

Table reflection
-----

In [19]:
#
# Generación una tabla leyendo el estado de la base de datos
#
# Proceso normal:
#     1. Se declara la tabla
#     2. Se emite el DDL
#
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.commit()  # confirmar los cambios hasta aca

2022-09-13 11:55:56,394 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:56,395 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-09-13 11:55:56,396 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ()
2022-09-13 11:55:56,397 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
#
# En este caso se hace de forma inversa
#
some_table = Table(
    "some_table",
    metadata_obj,
    autoload_with=engine,
)

2022-09-13 11:55:56,452 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 11:55:56,453 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-09-13 11:55:56,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:56,455 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-09-13 11:55:56,456 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-09-13 11:55:56,457 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-09-13 11:55:56,457 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:56,458 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-09-13 11:55:56,459 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-13 11:55:56,459 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [21]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)